In [121]:
from lofar_lst import *
from lofar_planner import *
import os 
from datetime import datetime
from datetime import timedelta

['lofar_lst.py',
 '.DS_Store',
 'overall_planner.ipynb',
 '__pycache__',
 'obs_table.csv',
 '.ipynb_checkpoints',
 'lofar_planner.py',
 'GUI_testing',
 'data',
 'targets_of_interest.csv']

In [5]:
# --- Observation Window UTC --- 

start = '2022-07-08 07:00'; end = '2022-07-08 10:00'

UTC_alpha = 1
UTC_beta = 2

station_timezone(end, UTC_alpha)
LST_window(start, end)


 LST Window for UTC 07:00 to 10:00 on the 2022-07-08 is:

 I-LOFAR: 1:33-4:33
LOFAR-SE: 2:52-5:53

 LST difference of ~79 minutes between stations


In [6]:
def dataframe_formatting(fraction_of_ot): 

    """
    Inputs: 
    fraction_of_ot - filters out the percentage of observing time for that falls under a given requirement e.g. 0.5 = 50%

    Outputs: 
    A csv. with possible targets during the given observation window dictated by the parameters of the 'setup' function. 

    """

    combined_targets_dataframe = []
    # --- Organising Dataframes --- 
    combined_targets_dataframe = pd.merge(Station_Alpha[0], Station_Beta[0], on = "target name") # -- merges on title column 
    combined_targets_dataframe = combined_targets_dataframe.rename(columns={"ever observable_x": "Ever Observable(%s)" % observatory_1.name, "ever observable_y": "Ever Observable(%s)" % observatory_2.name, "always observable_x" : "Always Observable(%s)" % observatory_1.name, "always observable_y" : "Always Observable(%s)" % observatory_2.name, "fraction of time observable_x":"Fraction of observable time (%s)" % observatory_1.name, "fraction of time observable_y":"Fraction of observable time (%s)" % observatory_2.name, "target name": "Name"}) # -- renaming cols. 
    combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Ever Observable(%s)' % observatory_1.name] == True]; combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Ever Observable(%s)' % observatory_2.name] == True] # -- removing targets that can't be observed during the window. 
    combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Fraction of observable time (%s)' % observatory_1.name] > fraction_of_ot];  combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Fraction of observable time (%s)' % observatory_2.name] > fraction_of_ot] 
    combined_targets_dataframe = pd.merge(combined_targets_dataframe, Station_Beta[3], on = "Name") # - merges on target name. 
    combined_targets_dataframe.sort_values(by = 'RA', ascending = True, inplace = True) # - sorting values by ascending right ascension. 

    combined_targets_dataframe = combined_targets_dataframe[['Name', 'RA', 'DEC', 'Ever Observable(%s)' % observatory_1.name, 'Always Observable(%s)' % observatory_1.name, 'Ever Observable(%s)' % observatory_2.name, 'Always Observable(%s)' % observatory_2.name, 'Fraction of observable time (%s)' % observatory_1.name , 'Fraction of observable time (%s)' % observatory_2.name]]

    print('Number of possible targets:', len(combined_targets_dataframe))
    # --- Saving .csv ---
    combined_targets_dataframe.to_csv('data/target-lists/combined/Combined-Targets-%s.csv' % (Station_Alpha[2][2:8]), header = True, index  = False)

    return(combined_targets_dataframe)

In [37]:
# %%capture

Station_Alpha = []
Station_Beta = []

observatory_1 = ILOFAR
observatory_2 = ONSULA
catalog = 'targets_of_interest.csv'

# --- Computing Dataframes --- 
Station_Alpha = setup(station_timezone(start, UTC_alpha), station_timezone(end, UTC_alpha), 30, 90, 20, 0.016, observatory_1, catalog, True)
Station_Beta  = setup(station_timezone(start, UTC_beta), station_timezone(end, UTC_beta), 30, 90, 20,  0.016, observatory_2, catalog, False)

observation_table = dataframe_formatting(0.8)
observation_table.to_csv('obs_table.csv')

['2022-07-08 08:00:00.000' '2022-07-08 11:00:00.000']
Number of objects that do not meet the criteria of both stations: 273
Number of previously observed targets: 24
Number of objects avalible for observation: 546
['2022-07-08 09:00:00.000' '2022-07-08 12:00:00.000']
Number of objects that do not meet the criteria of both stations: 273
Number of previously observed targets: 24
Number of objects avalible for observation: 546
Number of possible targets: 546


In [136]:
def find_nearest(array, value): # - Function to find the nearest value in a given array. 
    array = np.asarray(array); idx = (np.abs(array - value)).argmin()
    return idx

base_lst = 2.5 # - LST that is best suited to both stations, usually half-way between both. 
target_numbers = 12; step = 0.25 # - The amount of targets to be observed along with the amount of targets 
idx_array = []

for i in range(0, target_numbers): # - Find targets that are visible and avalible at around 15 min intervals in the observation window. 
    trgt_idx = find_nearest(observation_table['RA'], base_lst)
    base_lst += step 
    if trgt_idx in idx_array: 
        idx_array.append(trgt_idx + 1)
    else: 
        idx_array.append(trgt_idx)

trgt_tbl = observation_table.iloc[idx_array]

In [132]:
ra = (trgt_tbl['RA']).to_list(); dec = (trgt_tbl['DEC']).to_list()
trgt_coords = SkyCoord(ra*u.hourangle, dec*u.degree)
names = trgt_tbl['Name'].to_list()

cur_time = datetime.strptime('07:03', '%H:%M')
cur_time_str = cur_time.strftime('%H:%M')
trgt_duration = 16 

print('Name Time RA DEC freqrng dur ')
for i in range(0, len(ra)): 
    print(names[i], cur_time_str, '{:.6f}'.format(trgt_coords[i].ra.radian), '{:.6f}'.format(trgt_coords[i].dec.radian), '110e6:190.2e6', '15m')
    cur_time = cur_time + timedelta(minutes=trgt_duration)
    cur_time_str = cur_time.strftime('%H:%M')

Name Time RA DEC freqrng dur 
TIC249945230 07:03 0.677366 0.842536 110e6:190.2e6 15m
TIC302773669 07:19 0.740658 1.252328 110e6:190.2e6 15m
TIC251384275 07:35 0.780515 1.025123 110e6:190.2e6 15m
TIC82010657 07:51 0.850903 1.051355 110e6:190.2e6 15m
TIC285677945 08:07 0.926752 0.881375 110e6:190.2e6 15m
TIC104208182 08:23 0.951242 0.889335 110e6:190.2e6 15m
TIC87090944 08:39 1.020784 1.029191 110e6:190.2e6 15m
TIC267694283 08:55 1.103572 0.937016 110e6:190.2e6 15m
TIC103448870 09:11 1.166491 1.274504 110e6:190.2e6 15m
TIC9837861 09:27 1.231305 0.947575 110e6:190.2e6 15m
TIC321041369 09:43 1.425429 0.994203 110e6:190.2e6 15m
TIC311121985 09:59 1.493675 0.864173 110e6:190.2e6 15m
